# ANALYSIS OF VIDEO GAMES STORE DATABASE USING PYTHON AND POSTGRESQL

In [268]:
import pandas as pd
import psycopg2
import warnings

In [269]:
conn_string = "host='localhost' dbname='video_games' user='postgres' password='56067031'"
conn = psycopg2.connect(conn_string)
warnings.filterwarnings('ignore', category=Warning)

### PRE DECLARING SCHEMA NAME SO THAT IT IS EASIER TO QUERY

In [270]:
schema_name = 'video_games'

## TABLE DETAILS

#### 1. GAME TABLE

In [271]:
pd.read_sql_query(f"SELECT * FROM {schema_name}.{game}", conn).set_index('id').head()

,genre_id,game_name
id,,
1,11,98 Koshien
2,8,.hack//G.U. Vol.1//Rebirth
3,8,.hack//G.U. Vol.2//Reminisce
4,8,.hack//G.U. Vol.2//Reminisce (jp sales)
5,8,.hack//G.U. Vol.3//Redemption


#### 2. GAME PLATFORM DETAILS TABLE

In [272]:
pd.read_sql_query(f"SELECT * FROM {schema_name}.{game_platform}", conn).set_index('id').head()

,game_publisher_id,platform_id,release_year
id,,,
1,8564,4,2007
2,9852,4,2007
3,11063,7,2006
4,9065,15,2011
5,9544,15,2011


#### 3. GAME PUBLISHER DETAILS TABLE

In [273]:
pd.read_sql_query(f"SELECT * FROM {schema_name}.{game_publisher}", conn).set_index('id').head()

,game_id,publisher_id
id,,
1,10866,369
2,9244,369
3,5464,369
4,10868,369
5,7282,369


#### 4. GENRE DETAILS TABLE

In [274]:
pd.read_sql_query(f"SELECT * FROM {schema_name}.{genre}", conn).set_index('id').head()

,genre_name
id,
1,Action
2,Adventure
3,Fighting
4,Misc
5,Platform


#### 5. PUBLISHER DETAILS TABLE

In [275]:
pd.read_sql_query(f"SELECT * FROM {schema_name}.{publisher}", conn).set_index('id').head()

,publisher_name
id,
1,10TACLE Studios
2,1C Company
3,20th Century Fox Video Games
4,2D Boy
5,3DO


#### 6. REGION DETAILS TABLE

In [276]:
pd.read_sql_query(f"SELECT * FROM {schema_name}.{region}", conn).set_index('id').head()

,region_name
id,
1,North America
2,Europe
3,Japan
4,Other


#### 7. REGION SALES DETAILS TABLE

In [277]:
pd.read_sql_query(f"SELECT * FROM {schema_name}.{region_sales}", conn).set_index('region_id').head()

,game_platform_id,num_sales
region_id,,
1,50,3.50
1,51,1.43
1,52,0.51
1,53,0.27
1,54,0.48


#### 8. PLATFORM DETAILS TABLE

In [278]:
pd.read_sql_query(f"SELECT * FROM {schema_name}.{platform}", conn).set_index('id').head()

,platform_name
id,
1,Wii
2,NES
3,GB
4,DS
5,X360


## DATA ANLYSIS QUESTIONS
#### 1. Who is the top game publisher based on the number of games published?

In [279]:
pd.read_sql_query(f"""
SELECT gp.publisher_id as "Publisher ID", pub.publisher_name as "Publisher Name", COUNT(DISTINCT gp.game_id) as "Number of Games Published"
FROM {schema_name}.game_publisher as gp
INNER JOIN {schema_name}.publisher as pub
ON gp.publisher_id = pub.id
GROUP BY publisher_id, pub.publisher_name
ORDER BY COUNT(DISTINCT game_id) DESC
LIMIT 1;
""", conn).set_index('Publisher ID')

,Publisher Name,Number of Games Published
Publisher ID,,
352,Namco Bandai Games,774


#### 2. Which platform has the highest total sales across all regions?

In [280]:
pd.read_sql_query(f"""
SELECT gp.platform_id as "Platform Id", plf.platform_name as "Platform Name", SUM(rs.num_sales) as "Total Sales"
FROM {schema_name}.game_platform as gp
INNER JOIN {schema_name}.platform as plf
ON gp.platform_id = plf.id
INNER JOIN {schema_name}.region_sales as rs
ON gp.id = rs.game_platform_id
GROUP BY platform_id, plf.platform_name
ORDER BY SUM(rs.num_sales) DESC
LIMIT 1;
""", conn).set_index('Platform Id')

,Platform Name,Total Sales
Platform Id,,
7,PS2,1233.56


#### 3. How many games are categorized under each genre?

In [281]:
pd.read_sql_query(f"""
SELECT gn.id as "Genre ID", gn.genre_name as "Genre Name", COUNT(gm.game_name) as "Number of Games"
FROM {schema_name}.genre as gn
INNER JOIN {schema_name}.game as gm
ON gn.id = gm.genre_id
GROUP BY gn.id, gn.genre_name
ORDER BY COUNT(gm.game_name) DESC;
""", conn).set_index('Genre ID')

,Genre Name,Number of Games
Genre ID,,
1,Action,1900
11,Sports,1366
4,Misc,1314
8,Role-Playing,1206
2,Adventure,1038
9,Shooter,806
7,Racing,761
10,Simulation,714
3,Fighting,605


#### 4. Which region has the highest total number of sales?

In [282]:
pd.read_sql_query(f"""
SELECT rg.id as "Region ID", rg.region_name as "Region Name", SUM(rs.num_sales) as "Total Number of Sales"
FROM {schema_name}.region as rg
INNER JOIN {schema_name}.region_sales as rs
ON rg.id = rs.region_id
GROUP BY rg.id, rg.region_name
ORDER BY SUM(rs.num_sales) DESC
LIMIT 1;
""", conn).set_index('Region ID')

,Region Name,Total Number of Sales
Region ID,,
1,North America,4335.07


#### 5. Which game has the highest sales in North America?

In [283]:
# We know from the region table that North America has id = 1
pd.read_sql_query(f"""
SELECT gm.id as "Game ID", gm.game_name as "Game Name", gpf.game_publisher_id as "Publisher ID", rs.region_id as "Region ID", SUM(rs.num_sales) as "Total Sales"
FROM {schema_name}.game as gm
INNER JOIN {schema_name}.game_publisher as gp
ON gm.id = gp.game_id
INNER JOIN {schema_name}.game_platform as gpf
ON gp.id = gpf.game_publisher_id
INNER JOIN {schema_name}.region_sales as rs
ON gpf.id = rs.game_platform_id
WHERE rs.region_id = 1
GROUP BY gm.id,gm.game_name, gpf.game_publisher_id, rs.region_id
ORDER BY SUM(rs.num_sales) DESC
LIMIT 1;
""", conn).set_index('Game Name')

,Game ID,Publisher ID,Region ID,Total Sales
Game Name,,,,
Wii Sports,10866,1,1,41.49


#### 6. How many games were released in each year?

In [284]:
pd.read_sql_query(f"""
SELECT gp.release_year as "Release Year", COUNT(DISTINCT gp.game_publisher_id) as "Number of Games Released"
FROM {schema_name}.game_platform as gp
GROUP BY gp.release_year
ORDER BY COUNT(*) DESC;
""", conn).set_index('Release Year')

,Number of Games Released
Release Year,
2009,1013
2008,1051
2010,914
2007,900
2011,787
2006,777
2005,678
2002,645
2003,560


#### 7. Which is the popularity of different game genres based on number of sales ?

In [285]:
pd.read_sql_query(f"""
SELECT gn.genre_name as "Genre Name", SUM(rs.num_sales) as "Total Number of Sales"
FROM {schema_name}.genre as gn
INNER JOIN {schema_name}.game as gm
ON gn.id = gm.genre_id
INNER JOIN {schema_name}.game_publisher as gp
ON gm.id = gp.game_id
INNER JOIN {schema_name}.game_platform as gpf
ON gp.id = gpf.game_publisher_id
INNER JOIN {schema_name}.region_sales as rs
ON gpf.id = rs.game_platform_id
GROUP BY gn.genre_name
ORDER BY SUM(rs.num_sales) DESC;
""", conn).set_index('Genre Name')

,Total Number of Sales
Genre Name,
Action,1722.05
Sports,1308.83
Shooter,1025.75
Role-Playing,924.08
Platform,828.82
Misc,796.83
Racing,729.79
Fighting,444.08
Simulation,389.88


#### 8. Which publisher has the highest total sales in each region?

In [286]:
x = pd.DataFrame()
for i in [1,2,3,4]:
    x = pd.concat([x,pd.read_sql_query(f"""
    SELECT rs.region_id as "Region ID",rg.region_name as "Region Name",
    SUM(rs.num_sales) as "Max Region Sales",gpf.game_publisher_id as "PubID", pb.publisher_name as "Publisher Name",
    gm.id as "Game ID", gm.game_name as "Game Name"
    FROM {schema_name}.game as gm
    INNER JOIN {schema_name}.game_publisher as gp
    ON gm.id = gp.game_id
    INNER JOIN {schema_name}.game_platform as gpf
    ON gp.id = gpf.game_publisher_id
    INNER JOIN {schema_name}.region_sales as rs
    ON gpf.id = rs.game_platform_id
    INNER JOIN {schema_name}.publisher as pb
    ON gp.publisher_id = pb.id
    INNER JOIN {schema_name}.region as rg
    ON rs.region_id = rg.id
    WHERE rs.region_id = {i}
    GROUP BY gm.id,gm.game_name, gpf.game_publisher_id, rs.region_id, pb.publisher_name, rg.region_name
    ORDER BY SUM(rs.num_sales) DESC
    LIMIT 1;
    """, conn)])
x = x.reset_index(drop=True).set_index('Region ID')
x

,Region Name,Max Region Sales,PubID,Publisher Name,Game ID,Game Name
Region ID,,,,,,
1,North America,41.49,1,Nintendo,10866,Wii Sports
2,Europe,29.02,1,Nintendo,10866,Wii Sports
3,Japan,10.55,5,Nintendo,7282,Pokemon Red/Pokemon Blue
4,Other,10.72,18,Take-Two Interactive,3665,Grand Theft Auto: San Andreas


#### 9. Which are the top 10 publishers who have highest total number of sales for all their games included?

In [287]:
pd.read_sql_query(f"""
SELECT pb.publisher_name as "Publisher Name",SUM(rs.num_sales) as "TOTAL SALES"
FROM {schema_name}.publisher as pb
INNER JOIN {schema_name}.game_publisher as gp
ON pb.id = gp.publisher_id
INNER JOIN {schema_name}.game_platform as gpf
ON gp.id = gpf.game_publisher_id
INNER JOIN {schema_name}.region_sales as rs
ON gpf.id = rs.game_platform_id
GROUP BY pb.publisher_name
ORDER BY SUM(rs.num_sales) DESC
LIMIT 10;
""", conn).set_index('Publisher Name')

,TOTAL SALES
Publisher Name,
Nintendo,1784.56
Electronic Arts,1096.45
Activision,721.06
Sony Computer Entertainment,607.27
Ubisoft,473.33
Take-Two Interactive,399.45
THQ,340.32
Konami Digital Entertainment,278.37
Sega,270.68


#### 10. What is the total number of games sold for 2D Boy, 3DO, Yumedia,Zenrin adn Zoo Digital Publishing ?

In [288]:
pd.read_sql_query(f"""
SELECT pb.publisher_name as "Publisher Name",SUM(rs.num_sales) as "TOTAL SALES"
FROM {schema_name}.publisher as pb
INNER JOIN {schema_name}.game_publisher as gp
ON pb.id = gp.publisher_id
INNER JOIN {schema_name}.game_platform as gpf
ON gp.id = gpf.game_publisher_id
INNER JOIN {schema_name}.region_sales as rs
ON gpf.id = rs.game_platform_id
WHERE pb.publisher_name IN ('2D Boy','3DO','Yumedia','Zenrin','Zoo Digital Publishing')
GROUP BY pb.publisher_name
""", conn).set_index('Publisher Name')

,TOTAL SALES
Publisher Name,
Zoo Digital Publishing,12.89
Yumedia,0.06
Zenrin,0.06
3DO,10.15
2D Boy,0.04
